In [1]:
import random
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, Embedding, Dropout, Bidirectional, GRU
from tensorflow.keras.optimizers import Adam
from gensim.models import Word2Vec
from tensorflow.keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

# Parameters for the model
embedding_dim = 100
latent_dim = 54

# Parameters
vocab_size = 1000
max_length = 100
epochs = 98
batch_size = 32
learning_rate = 0.01

# Load IMDB dataset
(x_train, _), (x_test, _) = imdb.load_data(num_words=vocab_size)
word_index = imdb.get_word_index()

# Create a reverse word index
reverse_word_index = {value + 3: key for key, value in word_index.items()}
reverse_word_index[0] = '<PAD>'
reverse_word_index[1] = '<START>'
reverse_word_index[2] = '<UNK>'
reverse_word_index[3] = '<UNUSED>'

# Convert sequences back to text
train_texts = [[reverse_word_index.get(i, '<UNK>') for i in sequence] for sequence in x_train]
test_texts = [[reverse_word_index.get(i, '<UNK>') for i in sequence] for sequence in x_test]
all_texts = train_texts + test_texts



17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [2]:
!pip uninstall numpy -y
!pip install numpy==1.26.4
!pip install --upgrade --force-reinstall gensim
!pip install --upgrade --force-reinstall tensorflow


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 81.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 22.2 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 113.3 MB/s

In [4]:

# Train a Word2Vec model
word2vec_model = Word2Vec(sentences=all_texts, vector_size=embedding_dim, window=5, min_count=1, workers=4, epochs=20)

# Convert the word2vec model to a dictionary
# Create an embedding matrix where each row index corresponds to a word index
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        try:
            embedding_vector = word2vec_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            # Words not found in the embedding index will be all zeros
            pass


In [5]:

def sequence_to_text(sequence):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in sequence])

def add_noise_to_text(sequence, noise_factor=0.1):
    noisy_sequence = []
    for i in sequence:
        if random.random() < noise_factor:
            # Randomly replace a word with another word
            noisy_sequence.append(random.randint(1, vocab_size - 1))
        else:
            noisy_sequence.append(i)
    return noisy_sequence

# Add noise to the data
noisy_x_train = [add_noise_to_text(seq) for seq in x_train]
noisy_x_test = [add_noise_to_text(seq) for seq in x_test]

# Convert to padded sequences
noisy_x_train_padded = pad_sequences(noisy_x_train, maxlen=max_length, padding='post')
noisy_x_test_padded = pad_sequences(noisy_x_test, maxlen=max_length, padding='post')

# Example of noisy text
print("Original:", sequence_to_text(x_train[0]))
print("Noisy:", sequence_to_text(noisy_x_train[0]))


Original: ? in with i like horrible business ? ? would killer ? which ? <START> going at fun <UNK> film make like lame character has novel ? a all final sense <UNK> real <START> find character nothing ? second perhaps they <START> find ? ? this city an br overall <START> horror has i <UNUSED> should ? was in with <START> ? ? despite <START> with their people is i like horrible an well it br ? <START> with this genre this is i taken that ? <UNK> she sex is and house and after <UNK> <START> ? ? i final which ? be <START> does is i an annoying <UNK> film where if at man it's film ? be <UNUSED> with is comedy you than some <UNK> in perfect i get ? and <START> think plot ? it fun <START> ? the ? <UNK> sequence at their like horrible wanted on getting night just the <START> ? ? br any other <START> couple it someone then he ? more on why <UNUSED> can't ? that <START> family with for still wanted on final <UNK> such his ? that if at you interesting how film any <START> family would i an ? oth

In [6]:
# Encoder
input_text = Input(shape=(max_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(input_text)
encoder_output = Bidirectional(LSTM(2*latent_dim, return_sequences=False))(encoder_embedding)
encoder_output = Dropout(0.15)(encoder_output)
encoder_output = Dense(latent_dim, activation='relu')(encoder_output)

# Decoder
decoder_input_seq = Input(shape=(max_length - 1,))
decoder_embedding = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(decoder_input_seq)
decoder_input = RepeatVector(max_length)(encoder_output)
decoder_lstm = LSTM(latent_dim, return_sequences=True)(decoder_input)
decoder_lstm = Dropout(0.15)(decoder_lstm)
decoder_lstm = LSTM(2*latent_dim, return_sequences=True)(decoder_lstm)
decoder_output = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_lstm)


# Compile and train with MSE loss
autoencoder = Model([input_text, decoder_input_seq], decoder_output)
autoencoder.compile(optimizer=Adam(learning_rate=learning_rate, clipnorm=1.0), loss='sparse_categorical_crossentropy')

# # First, pad the sequences to max_length
x_train_padded = pad_sequences(x_train, maxlen=max_length, padding='post')
x_test_padded = pad_sequences(x_test, maxlen=max_length, padding='post')

# y_train and y_test are simply the padded original sequences
y_train = x_train_padded
y_test = x_test_padded

# Prepare decoder_input_data by shifting the sequences by one position
decoder_input_train = np.zeros((x_train_padded.shape[0], max_length - 1), dtype=int)
decoder_input_test = np.zeros((x_test_padded.shape[0], max_length - 1), dtype=int)

for i in range(len(x_train_padded)):
    decoder_input_train[i] = x_train_padded[i, 1:]  # Drop the first word

for i in range(len(x_test_padded)):
    decoder_input_test[i] = x_test_padded[i, 1:]  # Drop the first word


In [7]:
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 100)  │    100,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 216)       │    180,576 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 216)       │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 54)        │     11,718 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 100, 54)   │          0 │ dense[0][0]       │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 100, 54)   │     23,544 │ repeat_vector[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 100, 54)   │          0 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 100, 108)  │     70,416 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 99)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 100, 1000) │    109,000 │ lstm_2[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 495,254 (1.89 MB)

 Trainable params: 395,254 (1.51 MB)

 Non-trainable params: 100,000 (390.62 KB)

In [8]:
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

def custom_lr_scheduler(epoch, lr):
    # Decrease learning rate by 0.1 factor every 5 epochs
    if epoch % 5 == 0 and epoch != 0:
        lr = lr * 0.1
    return lr

# Define the callback
lr_scheduler = LearningRateScheduler(custom_lr_scheduler)
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Train the model
autoencoder.fit([noisy_x_train_padded, decoder_input_train], np.expand_dims(y_train, -1),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=([noisy_x_test_padded, decoder_input_test], np.expand_dims(y_test, -1)),
          callbacks=[early_stopping, lr_scheduler])



Epoch 1/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 45ms/step - loss: 4.8980 - val_loss: 4.8065 - learning_rate: 0.0100
Epoch 2/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 44ms/step - loss: 4.7909 - val_loss: 4.8234 - learning_rate: 0.0100
Epoch 3/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - loss: 4.7838 - val_loss: 4.8033 - learning_rate: 0.0100
Epoch 4/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - loss: 4.7877 - val_loss: 4.8023 - learning_rate: 0.0100
Epoch 5/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 43ms/step - loss: 4.7856 - val_loss: 4.8148 - learning_rate: 0.0100
Epoch 6/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - loss: 4.7717 - val_loss: 4.7938 - learning_rate: 1.0000e-03
Epoch 7/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - loss: 4.7594 - val_loss: 4.6633 - learning_rate: 1.0000e-03
Epoch 8/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 43ms/step - loss: 4.6406 - val_loss: 4.6222 - learning_rate: 1.0000e-03
Epoch 9/98
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - loss: 4.6019 - val_loss: 4.6017 - le

In [9]:
def sample(preds, temperature=1.0):
    # Convert to array and prevent numerical issues with very small numbers
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature  # Adjust by temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)  # Softmax
    probas = np.random.multinomial(1, preds, 1)  # Sample from the softmax distribution
    return np.argmax(probas)

In [10]:
import tensorflow as tf
# Prepare the decoder input for prediction
decoder_input_test = np.zeros((noisy_x_test_padded.shape[0], max_length - 1), dtype=int)
for i in range(len(x_test_padded)):
    decoder_input_test[i] = x_test_padded[i, 1:]  # Drop the first word

# Predicting denoised text
denoised_texts = autoencoder.predict([noisy_x_test_padded[:10], decoder_input_test[:10]])

# Function to convert sequences back to text
def decode_denoised_sequence(sequence):
    indices = tf.math.argmax(sequence, axis=1).numpy()
    return ' '.join([reverse_word_index.get(word, '?') for word in indices])

def decode_sequence_with_sampling(prob_distributions, temperature=1.0):
    return ' '.join([reverse_word_index.get(sample(probs, temperature), '?') for probs in prob_distributions])

# Decoding the denoised sequences
for i in range(10):
    print("Original:", sequence_to_text(x_test[i]))
    print("Noisy:", sequence_to_text(noisy_x_test_padded[i]))
    print("Generated:", decode_sequence_with_sampling(denoised_texts[i], temperature=1))
    print("\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Original: ? murder both in have <UNUSED> easily of of ? ? <UNK> <START> boring the <START> again ? understand dead <START> over a ? ? ? of of br how where first lead ? make you ? in have movie not ? are role dark and where in true director and old just <UNK> not last i lot ? an he film ? based both in <UNUSED> easily
Noisy: ? murder both in have <UNUSED> easily of whether ? ? <UNK> <START> boring the <START> again that understand dead <START> more a ? ? ? of of br how where plus lead ? try that's ? in have movie not ? are role dark and where in true director and old just on not last i lot ? poor he ? ? based both in <UNUSED> easily ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Generated: <START> no this emotional still and please <UNK> ok there bad also <UNK> br which in a theater and in the of on head <UNK> br became overall and at <UNK> wouldn't do a be little is character the say while were <UNK> <UNK> were and and it good need s